In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("linear").getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [12]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
lr = LinearRegression(featuresCol='features',
    labelCol='label',predictionCol='prediction')

In [7]:
lrModel = lr.fit(training)

In [8]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [13]:
lrModel.intercept

0.14228558260358093

In [14]:
training_summary = lrModel.summary

In [15]:
training_summary.r2

0.027839179518600154

In [16]:
training_summary.rootMeanSquaredError

10.16309157133015

In [17]:
all_data =  spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [24]:
train,test = all_data.randomSplit([0.7,0.3])

In [25]:
training.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                501|
|   mean|0.25688882219498976|
| stddev| 10.317884030544564|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [26]:
train.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                348|
|   mean|0.15282515827725499|
| stddev| 10.438515353752507|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [28]:
test.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                153|
|   mean|0.49358264600788443|
| stddev|  10.06786250411804|
|    min| -23.51088409032297|
|    max| 26.903524792043335|
+-------+-------------------+



In [29]:
new_model = lr.fit(train)

In [30]:
test_results = new_model.evaluate(test)

In [32]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-22.405127189859677|
|-23.143240489036128|
|-20.465604063023363|
|-20.243558499840187|
|  -17.3038768489581|
|-18.977201065392563|
|-17.735783285150344|
| -17.33974264895044|
|-16.802850241953763|
|-17.602309827090316|
| -13.76573074397016|
|-12.981210371874571|
|-12.696237736364258|
|-11.558058240205069|
|-14.288701416079094|
|-15.667219405646428|
|-10.653496791912797|
| -9.778573530501493|
| -13.21843098261514|
|-12.682394460425675|
+-------------------+
only showing top 20 rows



In [33]:
test_results.rootMeanSquaredError

9.87912606126146

In [34]:
unlabled_data = training.select("features")

In [36]:
predictions = new_model.transform(unlabled_data)

In [37]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| 0.5968208159808763|
|(10,[0,1,2,3,4,5,...|-0.6060814329730505|
|(10,[0,1,2,3,4,5,...|0.17115861287852996|
|(10,[0,1,2,3,4,5,...| 1.4895978820027254|
|(10,[0,1,2,3,4,5,...| 1.2514065684108517|
|(10,[0,1,2,3,4,5,...|-1.3938952050468834|
|(10,[0,1,2,3,4,5,...| 1.2638902095642852|
|(10,[0,1,2,3,4,5,...|0.02139927784895672|
|(10,[0,1,2,3,4,5,...|  -2.19595678711483|
|(10,[0,1,2,3,4,5,...| 1.9861469056015022|
|(10,[0,1,2,3,4,5,...|-0.6269792338956078|
|(10,[0,1,2,3,4,5,...| 1.8928853200624338|
|(10,[0,1,2,3,4,5,...| 2.3291330934146726|
|(10,[0,1,2,3,4,5,...| 0.4081214472341527|
|(10,[0,1,2,3,4,5,...| 1.6210335298037983|
|(10,[0,1,2,3,4,5,...| -1.864415774005661|
|(10,[0,1,2,3,4,5,...|  -2.43224386117373|
|(10,[0,1,2,3,4,5,...| -1.573091229420159|
|(10,[0,1,2,3,4,5,...| 2.0948451511234825|
|(10,[0,1,2,3,4,5,...| 0.5762874760298276|
+----------